In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import json
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2021-10-21 19:19:19,-46.1927,168.8643,39.43,91,23,2.53
1,1,Jalu,LY,2021-10-21 19:19:20,29.0331,21.5482,73.45,42,0,8.72
2,2,Sao Joao Da Barra,BR,2021-10-21 19:19:20,-21.6403,-41.0511,69.89,75,78,8.95
3,3,Ponta Do Sol,PT,2021-10-21 19:19:20,32.6667,-17.1000,72.23,49,3,6.20
4,4,Thompson,CA,2021-10-21 19:19:20,55.7435,-97.8558,32.16,74,75,8.05


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of humidity
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity.
humidity = city_data_df["Humidity"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
hum_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(hum_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of cloudiness
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the cloudiness.
cloudiness = city_data_df["Cloudiness"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
cloud_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(cloud_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the cloudiness.
wind = city_data_df["Wind Speed"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
wind_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(wind_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 40


In [11]:
preferred_cities_df =  city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)].dropna()

In [12]:
preferred_cities_df.count()

City_ID       72
City          72
Country       72
Date          72
Lat           72
Lng           72
Max Temp      72
Humidity      72
Cloudiness    72
Wind Speed    72
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mataura,NZ,39.43,-46.1927,168.8643,
4,Thompson,CA,32.16,55.7435,-97.8558,
8,Kodiak,US,38.71,57.7900,-152.4072,
10,Tuktoyaktuk,CA,28.40,69.4541,-133.0374,
20,Suntar,RU,38.62,62.1444,117.6319,
24,Talnakh,RU,24.33,69.4865,88.3972,
25,Yeniseysk,RU,36.82,58.4497,92.1703,
28,Iqaluit,CA,33.53,63.7506,-68.5145,
33,Boden,SE,32.00,65.8252,21.6886,
34,Husavik,IS,31.91,66.0449,-17.3389,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
    #"location": "48.8566, 2.3522"}

In [15]:
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
#hotels = requests.get(base_url, params=params).json()
#print(json.dumps(hotels, indent=4, sort_keys=True))

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    #print(f"{params}  index {index}")
    # Use the search term: "lodging" and our latitude and longitude.
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mataura,NZ,39.43,-46.1927,168.8643,Ellie's Villa
4,Thompson,CA,32.16,55.7435,-97.8558,Burntwood Hotel
8,Kodiak,US,38.71,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
10,Tuktoyaktuk,CA,28.40,69.4541,-133.0374,End of The Road Ltd.
20,Suntar,RU,38.62,62.1444,117.6319,Гостевая квартира
...,...,...,...,...,...,...
514,Beaverlodge,CA,34.95,55.2166,-119.4361,Side Trax Inn
552,Nikel,RU,33.98,69.4128,30.2220,Severnoe Siyanie
556,Smithers,CA,37.72,54.7804,-127.1743,"Prestige Hudson Bay Lodge, Premier Collection"
558,Inta,RU,30.34,66.0317,60.1659,Apartments in Inta city


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))